In [1]:
# ___Cell no. 1___

import tweepy
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from wordcloud import WordCloud
import nltk 
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# ___Cell no. 2___

# We then import the credentials
import twitter_credentials as tc

ModuleNotFoundError: ignored

In [ ]:
# ___Cell no. 3___

# Create an authentication object of the AuthHandler class by passing in the credentials
auth = tweepy.OAuthHandler(tc.api_key, tc.api_secret_key)

# Set the access tokens to complete the authentication process
auth.set_access_token(tc.access_token, tc.access_token_secret)

In [ ]:
# ___Cell no. 4___

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# ___Cell no. 23___

search_words = "(coronavirus OR covid OR pandemic OR covid19 OR lockdown OR corona) AND (vaccine OR astrazeneca OR pfizer OR moderna OR vaccination)"

# Latitude, logitude and search radius(km) for Zambia and some surrounding regions
loc = " 0.0236,39.9062,300km"

# Search dates
date_since = "2021-01-29"
date_until = "2021-07-29"

# We also want to exclude retweets and replies as this may sway results
my_search = search_words + " -filter:retweets" + " -filter:replies"  

In [ ]:
# ___Cell no. 24___

# Use the tweepy Cursor method to access tweets from a specified region and between certain dates
tweets = tweepy.Cursor(api.search,
                       #q='{} place: {kenya}'.format(my_search,place_id),
                       q=my_search,
                       lang="en",
                       tweet_mode='extended',
                       geocode=loc,
                       since=date_since,
                       until=date_until).items(1000)

In [ ]:
%%time

# ___Cell no. 25___

# Once again, this should take ~2 minutes to run if you are collecting 1000 tweets
tweet_info = [[tweet.id_str,tweet.created_at,tweet.user.location,tweet.full_text] for tweet in tweets]

# Put our data into a dataframe 
df_new = pd.DataFrame(data=tweet_info, columns=['tweet_id_str','date_time','location','tweet_text'])

# Have a quick look at the dataframe
df_new

CPU times: user 429 ms, sys: 24.6 ms, total: 454 ms
Wall time: 40 s


,tweet_id_str,date_time,location,tweet_text
0,1420505284691193866,2021-07-28 22:03:46,"Nairobi, Kenya",States that stockpile COVID-19 vaccines have c...
1,1420500621787078661,2021-07-28 21:45:14,"Nairobi, Kenya",Samia Suluhu Rolls Out COVID-19 Vaccination Dr...
2,1420491320544669702,2021-07-28 21:08:16,"Nairobi, Kenya",Covid-19 vaccine companies may have to prepare...
3,1420489668555051012,2021-07-28 21:01:43,"Nairobi, Kenya",Why should COVID-19 vaccines expire in some co...
4,1420488617181523970,2021-07-28 20:57:32,"Nairobi, Kenya",Whenever i hear a country has destroyed COVID-...
...,...,...,...,...
386,1417817240003559430,2021-07-21 12:02:26,"Nairobi, Kenya",Cisco SD-Access May Just Be the Vaccine You Ne...
387,1417812097560453120,2021-07-21 11:42:00,Nairobi,Pfizer announces deal to manufacture Covid-19 ...
388,1417811819402604544,2021-07-21 11:40:54,Nairobi,#UGANDA: Experts confirm #Ugandans got water f...
389,1417805423525179394,2021-07-21 11:15:29,"Nairobi, Kenya",Former Prime Minister Raila Odinga has called ...


In [ ]:
df_new.to_csv('kenya.csv')

In [ ]:
df = pd.read_csv('kenya.csv')
df

,Unnamed: 0,tweet_id_str,date_time,location,tweet_text
0,0,1420505284691193866,2021-07-28 22:03:46,"Nairobi, Kenya",States that stockpile COVID-19 vaccines have c...
1,1,1420500621787078661,2021-07-28 21:45:14,"Nairobi, Kenya",Samia Suluhu Rolls Out COVID-19 Vaccination Dr...
2,2,1420491320544669702,2021-07-28 21:08:16,"Nairobi, Kenya",Covid-19 vaccine companies may have to prepare...
3,3,1420489668555051012,2021-07-28 21:01:43,"Nairobi, Kenya",Why should COVID-19 vaccines expire in some co...
4,4,1420488617181523970,2021-07-28 20:57:32,"Nairobi, Kenya",Whenever i hear a country has destroyed COVID-...
...,...,...,...,...,...
386,386,1417817240003559430,2021-07-21 12:02:26,"Nairobi, Kenya",Cisco SD-Access May Just Be the Vaccine You Ne...
387,387,1417812097560453120,2021-07-21 11:42:00,Nairobi,Pfizer announces deal to manufacture Covid-19 ...
388,388,1417811819402604544,2021-07-21 11:40:54,Nairobi,#UGANDA: Experts confirm #Ugandans got water f...
389,389,1417805423525179394,2021-07-21 11:15:29,"Nairobi, Kenya",Former Prime Minister Raila Odinga has called ...


In [ ]:
# ___Cell no. 13___

def clean_text(text):
    
    """
    A function to clean the tweet text
    """
    #Remove hyper links
    text = re.sub(r'https?:\/\/\S+', ' ', text)
    
    #Remove @mentions
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    
    #Remove anything that isn't a letter, number, or one of the punctuation marks listed
    text = re.sub(r"[^A-Za-z0-9#'?!,.]+", ' ', text)
    return text

In [ ]:
# ___Cell no. 14___

# Apply the clean_text function to the 'tweet_text' column
df['tweet_text']=df['tweet_text'].apply(clean_text)

In [ ]:
df


,Unnamed: 0,tweet_id_str,date_time,location,tweet_text
0,0,1420505284691193866,2021-07-28 22:03:46,"Nairobi, Kenya",States that stockpile COVID 19 vaccines have c...
1,1,1420500621787078661,2021-07-28 21:45:14,"Nairobi, Kenya",Samia Suluhu Rolls Out COVID 19 Vaccination Dr...
2,2,1420491320544669702,2021-07-28 21:08:16,"Nairobi, Kenya",Covid 19 vaccine companies may have to prepare...
3,3,1420489668555051012,2021-07-28 21:01:43,"Nairobi, Kenya",Why should COVID 19 vaccines expire in some co...
4,4,1420488617181523970,2021-07-28 20:57:32,"Nairobi, Kenya",Whenever i hear a country has destroyed COVID ...
...,...,...,...,...,...
386,386,1417817240003559430,2021-07-21 12:02:26,"Nairobi, Kenya",Cisco SD Access May Just Be the Vaccine You Ne...
387,387,1417812097560453120,2021-07-21 11:42:00,Nairobi,Pfizer announces deal to manufacture Covid 19 ...
388,388,1417811819402604544,2021-07-21 11:40:54,Nairobi,#UGANDA Experts confirm #Ugandans got water fo...
389,389,1417805423525179394,2021-07-21 11:15:29,"Nairobi, Kenya",Former Prime Minister Raila Odinga has called ...


In [ ]:
# ___Cell no. 15___

for i,tweet in enumerate(df['tweet_text'].head(20)):
    print(i+1, tweet, '\n')

1 States that stockpile COVID 19 vaccines have committed to sharing their leftovers with low income and middle income countries. This is inexplicable since the efficacy of the vaccine will be low, the countries will be forced to dump again the vaccines. #VaccineEquity  

2 Samia Suluhu Rolls Out COVID 19 Vaccination Drive  

3 Covid 19 vaccine companies may have to prepare booster shots every year to handle new mutations. In fact, they have this in plan. Enzolytics ENZC is the only company preparing a one time shot, not just for covid, but for all major viruses known to terrorise mankind. 

4 Why should COVID 19 vaccines expire in some countries while other poorer nations are suffering from vaccine inadequacy due to Vaccine Apartheid that is witnessed globally? Is it that hard to transport? #VaccineEquity blackstock  

5 Whenever i hear a country has destroyed COVID 19 Vaccines i rethink, I'm quite sure if it is an African country it cannot be caused by the notion of vaccine hesitancy 

In [ ]:
# ___Cell no. 24___

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# ___Cell no. 25___

from simpletransformers.classification import ClassificationModel

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random
import numpy as np
import torch
from sklearn.model_selection import KFold

import logging
from pathlib import Path

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
# ___Cell no. 26___

df2 = pd.read_csv('labelled_data_cleaned.csv')
df2

,tweet_text,label
0,Jesus Ive actually spent all the money I saved...,1
1,Coronavirus and throat cancer looking after yo...,1
2,Me sick at home in Houston w Covid symptoms ri...,1
3,Coronavirus pandemic prompts record drop in gl...,1
4,Coronavirus How can we make postpandemic citie...,1
...,...,...
4224,Was recently tested for #coronavirus not at al...,1
4225,"Despite the Lockdown, Crises, Vera Lynn We'll ...",1
4226,"Due to COVID and mandatory face masks, million...",0
4227,Something someone said to me in regards to me ...,0


In [ ]:
# ___Cell no. 27___

bi_df = df2[(df2.label==0) | (df2.label==2) ] # make it binary classification
bi_df.label.replace(2,1, inplace=True) # make it binary classification

pos_samples = bi_df[bi_df['label']==1]
neg_samples = bi_df[bi_df['label']==0].sample(len(pos_samples), random_state=42)

bal_bi_df = pd.concat([pos_samples, neg_samples])

bi_df['label'].value_counts()
bal_bi_df['label'].value_counts()

/opt/workshop/lib/python3.6/site-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


0    1527
1     747
Name: label, dtype: int64

1    747
0    747
Name: label, dtype: int64

In [ ]:
# ___Cell no. 28___

train_df, val_df = train_test_split(bi_df, test_size=0.2,  random_state=42)

In [ ]:
# ___Cell no. 29___

train_df['label'].value_counts()
val_df['label'].value_counts()

0    1213
1     606
Name: label, dtype: int64

0    314
1    141
Name: label, dtype: int64

In [ ]:
# ___Cell no. 30___

# Build the model

bert_model = ClassificationModel('bert',
                            'bert-base-cased',
                            num_labels=2,
                            use_cuda=False,
                            args={'overwrite_output_dir': True})

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
%%time

# ___Cell no. 31___

# Train the model 
bert_model.train_model(train_df=train_df, eval_df=val_df)

/opt/workshop/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:278: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/opt/workshop/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/opt/workshop/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



CPU times: user 1h 1min 27s, sys: 57.3 s, total: 1h 2min 24s
Wall time: 4min 19s


In [ ]:
# ___Cell no. 32___

# wrapper functions

def multi_F1(y_true, y_pred, average='macro'):
    return sklearn.metrics.f1_score(y_true=y_true, y_pred=y_pred, average=average)

def multi_classification_report(y_true, y_pred):
    return sklearn.metrics.classification_report(y_true=y_true, y_pred=y_pred)

In [ ]:
%%time

# ___Cell no. 33___

# Calculated and print out the f1 score

result, model_outputs, wrong_predictions = bert_model.eval_model(val_df, f1=multi_F1);
print('f1 score = ',result['f1'])

/opt/workshop/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:751: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



f1 score =  0.8751885886709643
CPU times: user 3min 42s, sys: 967 ms, total: 3min 43s
Wall time: 15.5 s


In [ ]:
%%time

# ___Cell no. 34___

# Calculated and print out the results in the classification report

result, model_outputs, wrong_predictions = bert_model.eval_model(val_df, report=multi_classification_report);
print('Classification Report: ', result['report'])


Classification Report:                precision    recall  f1-score   support

           0       0.92      0.89      0.91       314
           1       0.78      0.82      0.80       141

    accuracy                           0.87       455
   macro avg       0.85      0.86      0.85       455
weighted avg       0.87      0.87      0.87       455

CPU times: user 3min 49s, sys: 920 ms, total: 3min 49s
Wall time: 16 s


In [ ]:
# ___Cell no. 35___

# The text from your new tweets - here we are just using a small subset of the exisiting tweet_text column
# for demonstration purposes.

new_tweets = df['tweet_text'].values

In [ ]:
#After collecting, cleaning and pre-processing your tweets, you will extra# ___Cell no. 36___

# Obtain the model's sentiment predictions for your collected tweets

new_result = bert_model.predict(new_tweets)
print(new_result[0])


[0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 0
 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0
 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 0
 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0
 1 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1
 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0]


In [ ]:
type(new_result)

tuple

In [ ]:
df= pd.DataFrame(list(new_result))
df=df.transpose()
df

,0,1
0,0,"[1.8926091, -1.2198372]"
1,0,"[0.7175603, -0.4452549]"
2,0,"[1.6162598, -1.1184485]"
3,0,"[1.9858279, -1.175507]"
4,0,"[1.9838449, -1.2376429]"
...,...,...
386,1,"[-0.25104767, 0.3851862]"
387,1,"[-0.23137069, 0.30561644]"
388,1,"[-0.6217906, 0.851051]"
389,1,"[-0.013043813, 0.21963704]"
